In [1]:
import numpy as np
import pandas as pd

import ast
import random
import lmstudio as lms

from IPython.display import Markdown

In [2]:
raw_link = pd.read_csv('postprocess.csv')
raw_tthc = pd.read_csv('data/raw_tthc.csv', dtype={'maThuTuc': str})

In [3]:
cols = ['link', 'Mã thủ tục hành chính liên quan', 'Số quyết định', 'Tên thủ tục', 'Cấp thực hiện', 'Loại thủ tục', 'Lĩnh vực', 
        'Trình tự thực hiện', 'Cách thức thực hiện', 'Thành phần hồ sơ', 'Đối tượng thực hiện', 'Cơ quan thực hiện', 
        'Cơ quan có thẩm quyền', 'Địa chỉ tiếp nhận hồ sơ', 'Cơ quan được ủy quyền', 'Cơ quan phối hợp', 'Kết quả thực hiện', 
        'Căn cứ pháp lý', 'Yêu cầu, điều kiện thực hiện', 'Từ khóa', 'Mô tả']

In [4]:
raw_tthc.columns = cols

In [5]:
pattern = []

pattern.append("Bạn đang cố gắng xác định xem câu trả lời có hiểu sai ngữ cảnh và mục đích của câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có sự mâu thuẫn về mặt thực tế so với kiến thức liên quan hay không. Một số thông tin trong câu trả lời có thể đã được bịa ra.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có quá chung chung hoặc quá chi tiết so với mức độ cụ thể phù hợp để trả lời câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có thể được suy ra đúng từ kiến thức liên quan hay không.")

In [6]:
config = {"temperature": 0.7, "maxTokens": 512, "topPSampling": 0.9}

In [7]:
evaluate_template = \
"""Bạn sẽ đóng vai trò là một người đánh giá câu trả lời (answer judge). Với một câu hỏi và câu trả lời, mục tiêu của bạn là xác định xem câu trả lời được cung cấp có chứa thông tin không đúng sự thật hoặc thông tin ảo giác (hallucinated information) hay không. {pattern}

#Câu hỏi#: {question}

#Câu trả lời#: {answer}

#Đánh giá của bạn#:

Bạn nên cố gắng hết sức để xác định xem câu trả lời có chứa thông tin không đúng sự thật hoặc thông tin ảo giác hay không. Câu trả lời bạn đưa ra PHẢI là "Có" hoặc "Không".
"""

In [8]:
model = lms.llm("falcon-3-1b")

df = pd.DataFrame(columns=['link', 'cauTraLoiAoGiac', 'pattern', 'danhGia'])

for i in range(len(raw_link)):
    p = raw_link['pattern'][i]
    question = raw_link['cauHoi'][i]
    answer = raw_link['cauTraLoiAoGiac'][i]
    prompt = evaluate_template.format(pattern=pattern[p], question=question, answer=answer)

    try:
        completion = model.respond(prompt, config=config)
        output = str(completion)
    except Exception as e:
        output = ""
        print(e)
        break

    results = [raw_link['link'][i], answer, p, output]
    df.loc[i, ['link', 'cauTraLoiAoGiac', 'pattern', 'danhGia']] = results    

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

In [9]:
df.to_csv("data/falcon-3-1b.csv")

In [10]:
model = lms.llm("tiny-vicuna-1b")

df = pd.DataFrame(columns=['link', 'cauTraLoiAoGiac', 'pattern', 'danhGia'])

for i in range(len(raw_link)):
    p = raw_link['pattern'][i]
    question = raw_link['cauHoi'][i]
    answer = raw_link['cauTraLoiAoGiac'][i]
    prompt = evaluate_template.format(pattern=pattern[p], question=question, answer=answer)

    try:
        completion = model.respond(prompt, config=config)
        output = str(completion)
    except Exception as e:
        output = ""
        print(e)
        break

    results = [raw_link['link'][i], answer, p, output]
    df.loc[i, ['link', 'cauTraLoiAoGiac', 'pattern', 'danhGia']] = results    

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

Chat response error: Trying to keep the first 2159 tokens when context the overflows. However, the model is loaded with context length of only 2048 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input


In [11]:
df.to_csv("data/tiny-vicuna-1b.csv")

In [12]:
model = lms.llm("falcon-3-1b")

df = pd.DataFrame(columns=['link', 'cauTraLoi', 'danhGia'])

for i in range(len(raw_link)):
    question = raw_link['cauHoi'][i]
    answer = raw_link['cauTraLoi'][i]
    prompt = evaluate_template.format(pattern="", question=question, answer=answer)

    try:
        completion = model.respond(prompt, config=config)
        output = str(completion)
    except Exception as e:
        output = ""
        print(e)
        break

    results = [raw_link['link'][i], answer, output]
    df.loc[i, ['link', 'cauTraLoi', 'danhGia']] = results    

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

Chat response error: Trying to keep the first 5218 tokens when context the overflows. However, the model is loaded with context length of only 4096 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input


In [13]:
df.to_csv("data/falcon-3-1b_right.csv")

In [14]:
model = lms.llm("tiny-vicuna-1b")

df = pd.DataFrame(columns=['link', 'cauTraLoi', 'danhGia'])

for i in range(len(raw_link)):
    question = raw_link['cauHoi'][i]
    answer = raw_link['cauTraLoi'][i]
    prompt = evaluate_template.format(pattern="", question=question, answer=answer)

    try:
        completion = model.respond(prompt, config=config)
        output = str(completion)
    except Exception as e:
        output = ""
        print(e)
        break

    results = [raw_link['link'][i], answer, output]
    df.loc[i, ['link', 'cauTraLoi', 'danhGia']] = results    

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

Chat response error: Trying to keep the first 2115 tokens when context the overflows. However, the model is loaded with context length of only 2048 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input


In [15]:
df.to_csv("data/tiny-vicuna-1b_right.csv")